In [7]:
import psutil

def log_memory_usage():
    memory = psutil.virtual_memory()
    print(f"Used: {memory.used / 1e9:.2f} GB, Available: {memory.available / 1e9:.2f} GB")

In [8]:
!pip install langdetect
!pip install googletrans

In [9]:
pip install transformers


In [10]:
pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: torch in c:\users\this pc\anaconda3\envs\aidetector\lib\site-packages (2.4.1)



In [11]:
pip install protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install tiktoken

In [13]:
pip install sentencepiece


In [14]:
import sys
import os

sys.path.append(os.path.abspath("."))


In [ ]:
!pip install transformers langdetect googletrans==4.0.0-rc1 wikipedia-api langchain llama-index

import asyncio
from langdetect import detect
from googletrans import Translator
from transformers import pipeline
import wikipediaapi

# Initialize components
translator = Translator()
qa_pipeline = pipeline("question-answering", model="deepset/xlm-roberta-base-squad2")
wiki_api = wikipediaapi.Wikipedia('en')

# Detect language
def detect_language(text):
    return detect(text)

# Translate text asynchronously
async def translate_text_async(text, target_lang="en"):
    try:
        translation = await translator.translate(text, dest=target_lang)
        return translation.text
    except Exception as e:
        print(f"Translation error: {e}")
        return text

# Fetch relevant context using Wikipedia
def fetch_context(query):
    page = wiki_api.page(query)
    if page.exists():
        return page.summary[:1000]  # Use the first 1000 characters as context
    return "No relevant information found."

# Main QA chain
async def process_query_async(query):
    lang = detect_language(query)
    print(f"Detected language: {lang}")

    # Translate query if not in English
    if lang != "en":
        query = await translate_text_async(query, target_lang="en")
        print(f"Translated query: {query}")

    # Retrieve dynamic context
    context = fetch_context(query)
    print(f"Retrieved context: {context}")

    # Process query with QA model
    response = qa_pipeline({'context': context, 'question': query})
    answer = response.get('answer', "No answer found.")

    # Translate answer back to original language if needed
    if lang != "en":
        answer = await translate_text_async(answer, target_lang=lang)
    return answer

# Test the pipeline
async def main():
    user_query = "Qu'est-ce que l'intelligence artificielle?"  # French question
    response = await process_query_async(user_query)
    print(f"Final Answer: {response}")

await main()


In [3]:
import src.helper
print(src.helper)


<module 'src.helper' from 'c:\\Users\\This PC\\AI_DETECTOR\\src\\helper.py'>
